In [8]:
import numpy as np

T, H = 5, 4
hs = np.arange(1,T*H+1).reshape(T, H)
print(hs)
print(hs[0])
a  = np.array([0.8, 0.1, 0.03, 0.05, 0.02])
print(a)
ar = a.reshape(-1, 1)
print(ar)
ar = ar.repeat(H, axis=1)
print(ar.shape)
print(ar)

t = hs * ar
print(t.shape)
print(t)

c = np.sum(t, axis=0)
print(c.shape)
print(c)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]
 [17 18 19 20]]
[1 2 3 4]
[0.8  0.1  0.03 0.05 0.02]
[[0.8 ]
 [0.1 ]
 [0.03]
 [0.05]
 [0.02]]
(5, 4)
[[0.8  0.8  0.8  0.8 ]
 [0.1  0.1  0.1  0.1 ]
 [0.03 0.03 0.03 0.03]
 [0.05 0.05 0.05 0.05]
 [0.02 0.02 0.02 0.02]]
(5, 4)
[[0.8  1.6  2.4  3.2 ]
 [0.5  0.6  0.7  0.8 ]
 [0.27 0.3  0.33 0.36]
 [0.65 0.7  0.75 0.8 ]
 [0.34 0.36 0.38 0.4 ]]
(4,)
[2.56 3.56 4.56 5.56]


In [14]:
import numpy as np

N, T, H = 2, 5, 4
hs = np.arange(1,N*T*H+1).reshape(N, T, H)
print(hs)
a  = np.array([[0.8, 0.1, 0.03, 0.05, 0.02],
               [0.01, 0.02, 0.9, 0.05, 0.02]])

print(a)
ar = a.reshape(N, T, 1)
print(ar)
ar = ar.repeat(H, axis=2)
print(ar)

t = hs * ar
print(t)

c = np.sum(t, axis=1)
print(c.shape)
print(c)

[[[ 1  2  3  4]
  [ 5  6  7  8]
  [ 9 10 11 12]
  [13 14 15 16]
  [17 18 19 20]]

 [[21 22 23 24]
  [25 26 27 28]
  [29 30 31 32]
  [33 34 35 36]
  [37 38 39 40]]]
[[0.8  0.1  0.03 0.05 0.02]
 [0.01 0.02 0.9  0.05 0.02]]
[[[0.8 ]
  [0.1 ]
  [0.03]
  [0.05]
  [0.02]]

 [[0.01]
  [0.02]
  [0.9 ]
  [0.05]
  [0.02]]]
[[[0.8  0.8  0.8  0.8 ]
  [0.1  0.1  0.1  0.1 ]
  [0.03 0.03 0.03 0.03]
  [0.05 0.05 0.05 0.05]
  [0.02 0.02 0.02 0.02]]

 [[0.01 0.01 0.01 0.01]
  [0.02 0.02 0.02 0.02]
  [0.9  0.9  0.9  0.9 ]
  [0.05 0.05 0.05 0.05]
  [0.02 0.02 0.02 0.02]]]
[[[ 0.8   1.6   2.4   3.2 ]
  [ 0.5   0.6   0.7   0.8 ]
  [ 0.27  0.3   0.33  0.36]
  [ 0.65  0.7   0.75  0.8 ]
  [ 0.34  0.36  0.38  0.4 ]]

 [[ 0.21  0.22  0.23  0.24]
  [ 0.5   0.52  0.54  0.56]
  [26.1  27.   27.9  28.8 ]
  [ 1.65  1.7   1.75  1.8 ]
  [ 0.74  0.76  0.78  0.8 ]]]
(2, 4)
[[ 2.56  3.56  4.56  5.56]
 [29.2  30.2  31.2  32.2 ]]


In [15]:
import sys
sys.path.append("..")
from common.layers import Softmax
hs = np.array([
                [
                  [1,0,-1,1],
                  [-1,0,1,1],
                  [0,1,1,-1],
                  [1,1,0,-1],
                  [-1,0,1,0]
                ]
              ])
h = np.array([[1,0,-1,1]])
N, T, H = hs.shape
hr = h.reshape(N, 1, H).repeat(T, axis=1)
t = hs * hr
s = np.sum(t, axis=2)
print(s)
softmax = Softmax()
a = softmax.forward(s)
print(a)

[[ 3 -1 -2  0 -2]]
[[0.92457451 0.01693417 0.00622973 0.04603185 0.00622973]]


In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Softmax


class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None

    def forward(self, hs, a):
        N, T, H = hs.shape

        ar = a.reshape(N, T, 1).repeat(H, axis=1)
        t = hs * ar
        c = np.sum(t, axis=1)

        self.cache = (hs, ar)
        return c

    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2)

        return dhs, da


class AttentionWeight:
    def __init__(self):
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = None

    def forward(self, hs, h):
        N, T, H = hs.shape

        hr = h.reshape(N, 1, H).repeat(T, axis=1)
        t = hs * hr
        s = np.sum(t, axis=2)
        a = self.softmax.forward(s)

        self.cache = (hs, hr)
        return a

    def backward(self, da):
        hs, hr = self.cache
        N, T, H = hs.shape

        ds = self.softmax.backward(da)
        dt = ds.reshape(N, T, 1).repeat(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)

        return dhs, dh


class Attention:
    def __init__(self):
        self.params, self.grads = [], []
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None

    def forward(self, hs, h):
        a = self.attention_weight_layer.forward(hs, h)
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        return out

    def backward(self, dout):
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh


class TimeAttention:
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []

        for t in range(T):
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:,t,:])
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)

        return out

    def backward(self, dout):
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(T):
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])
            dhs_enc += dhs
            dhs_dec[:,t,:] = dh

        return dhs_enc, dhs_dec
